### **README**

# What this does

This script allows you to download a single merged results tile with no row limit, as one `txt` file. Use at your own risk. Known limitations:

- Script downloads a single merged tile, not an entire dashboard.
- Script only allows for downloading a merge tile with 2 merged queries. Tiles with >2 queries will likely error or output misleading results. 
- Script may cause your computer to crash or slow. Why? 
  - It merges unlimited results by creating a pandas dataframe for each merged query, then using `pd.merge()`
  - As a result, your computer has to hold each dataframe in memory (however large they may be), then create another dataframe with size `df1+df2` and also hold that in memory. 


# Notes
`DownloadUnlimitedMerged_OneFile(dash_id, tile_title)` accepts 2 inputs:

- `dash_id`(string) - the ID of the dashboard
- `tile_title`(string) - the title of the merged results tile

Output files will be named `tile_title.txt`.

---


If using colab version, you will need to input your api url, client_id, and client_secret in the `## colab inputs ##` section. 

```
os.environ['LOOKERSDK_BASE_URL'] = 'https://example.looker.com'
os.environ['LOOKERSDK_API_VERSION'] = '3.1'
os.environ['LOOKERSDK_CLIENT_ID'] = 'client id here'
os.environ['LOOKERSDK_CLIENT_SECRET'] = 'client secret here'
```

If using python (local) version, these values should be set for the sdk per either an `.ini` file or as an environment variable. See Python SDK documentation for more information. https://pypi.org/project/looker-sdk/

In [1]:
### colab specific ###
# install the sdk
!pip install looker-sdk
# set environ variables
import os

## colab inputs ##
os.environ['LOOKERSDK_BASE_URL'] = ''
os.environ['LOOKERSDK_API_VERSION'] = '3.1'
os.environ['LOOKERSDK_CLIENT_ID'] = ''
os.environ['LOOKERSDK_CLIENT_SECRET'] = ''

In [2]:
# will need these for downloading the file from colab
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from google.colab import files
from oauth2client.client import GoogleCredentials
# auth stuff for download
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [3]:
### import packages ###
# import and init
import looker_sdk
sdk = looker_sdk.init31("looker.ini")

import pandas as pd
import csv

In [40]:
## inputs ##

dash_id = ''
tile_title = ''


In [41]:
# get merge query id from dashboard 
data = sdk.search_dashboard_elements(dashboard_id=dash_id, title=tile_title )
mergeid = data[0].merge_result_id

# get source query ids from merge query
mergedata = sdk.merge_query(merge_query_id=mergeid)
source_qs = mergedata.source_queries
num_merged = len(source_qs)

# init some lists
listoffiles = []
left_on = []
right_on = []
elements_needed = ['model','view','fields','filters','sorts','limit','query_timezone']
lenneedd = len(elements_needed)

# start god loop
for x in range(num_merged): 
    final_body = {}  
    # get raw query body
    source_qids = source_qs[x].query_id
    query_body = sdk.query(query_id=source_qids)
    
    # change limit to unlimited 
    query_body['limit'] = '-1'

    # add needed elements in raw body to final body
    for y in range(lenneedd):
      try:
          new_el = query_body[elements_needed[y]]
      except:
          y+=1
      else:
          final_body[elements_needed[y]] = new_el

    # run unlimited query
    resultset = sdk.run_inline_query('csv', final_body)
    
    # save files in the format 'Fooq1.csv` and record name in list
    with open(tile_title + 'q'+str(x)+'.csv', 'w') as file:
              file.write(resultset)
    listoffiles.append(tile_title + 'q'+str(x)+'.csv')


In [43]:
# get join strings
lefton = mergedata.source_queries[1].merge_fields[0].source_field_name
righton = mergedata.source_queries[1].merge_fields[0].field_name

# normalize
lefton = lefton.lower()
righton = righton.lower()
lefton = lefton.replace('.','')
righton = righton.replace('.','')
lefton = lefton.replace('_','')
righton = righton.replace('_','')
lefton = lefton.replace(' ','')
righton = righton.replace(' ','')


In [44]:
# probably unnecessary
merge = 'null'

# create dataframe 1 & normalize
df1 = pd.read_table(listoffiles[0], delimiter =",",engine='python')
df1.columns = df1.columns.str.lower()
df1.columns = df1.columns.str.replace('.','')
df1.columns = df1.columns.str.replace('_','')
df1.columns = df1.columns.str.replace(' ','')

# create dataframe 2 & normalize
df2 = pd.read_table(listoffiles[1], delimiter =",",engine='python')
df2.columns = df2.columns.str.lower()
df2.columns = df2.columns.str.replace('.','')
df2.columns = df2.columns.str.replace('_','')
df2.columns = df2.columns.str.replace(' ','')

# the merge. left just like merged results
merge = pd.merge(df1, df2, 
                   left_on= lefton , 
                   right_on= righton ,
                   how='left')

Index(['ordersstatus', 'ordersuserid', 'ordersid', 'orderscount'], dtype='object')
Index(['usersid', 'usersstate', 'userscount'], dtype='object')


In [45]:
# create txt file
merge.to_csv(tile_title+'.txt',
      sep=',',
      header=False,
      index=False,
      quoting=csv.QUOTE_ALL,
      quotechar='"',
      doublequote=True,
      line_terminator='\n')

# colab only
from google.colab import files 
files.download('merge.txt')
# end colab

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>